In [94]:
import sys
import os
import datetime
import numpy as np
#Loading utils for required funcs
%run {os.path.join('..', 'src', 'utils.py')}

## RFM
##### Recency, Frequency, Monetary value (RFM) is a model used in marketing analysis that segments a company's consumer base by their purchasing patterns or habits. In particular, it evaluates customers Recency (how long ago they made a purchase), Frequency (how often they make purchases), and Monetary value (how much money they spend).


In [102]:
#read our dataset
rfm_data = pd.read_csv(os.path.join("..","data","potentials.csv"))
payment = pd.read_csv(os.path.join("..","data","payment.csv"))
satisfaction=pd.read_csv(os.path.join("..","data","satisfaction.csv"))

In [103]:
# Boolean condition to filter rows
condition = (rfm_data['is_promotion'] != 1) & (rfm_data['current_products_price'] > 0) & (rfm_data['membership_length'] > 0)

# Filter the DataFrame based on the condition
rfm_data = rfm_data[condition].copy()

In [104]:
#filling the NaN budget_value with 1
rfm_data['budget_value'].fillna(1, inplace=True)

In [105]:
# Sort the satisfaction dataframe by date in descending order
satisfaction_sorted = satisfaction.sort_values('satisfaction_date', ascending=False)

# Drop duplicate rows based on customer_id and provider_id, keeping only the first occurrence (which will be the latest date)
satisfaction_latest = satisfaction_sorted.drop_duplicates(['customer_id', 'provider_id'])

In [106]:
# Merge the dataframes on provider_id and customer_id using a left join
merged_df = rfm_data.merge(satisfaction[['provider_id', 'customer_id', 'value']], on=['provider_id', 'customer_id'], how='left')

# Add the satisfaction_status column to rfm_data_subs
rfm_data['satisfaction_status'] = merged_df['value']

In [107]:
'''note for me : all the dublicated provider_id's are missing information thus there is no double provider_id
# Find and display the duplicated rows based on 'provider_id' column
duplicated_rows = satisfaction_latest[satisfaction_latest.duplicated(subset='provider_id', keep=False)]

# Print the duplicated rows
duplicated_rows
'''

"note for me : all the dublicated provider_id's are missing information thus there is no double provider_id\n# Find and display the duplicated rows based on 'provider_id' column\nduplicated_rows = satisfaction_latest[satisfaction_latest.duplicated(subset='provider_id', keep=False)]\n\n# Print the duplicated rows\nduplicated_rows\n"

In [108]:
'''
provider_counts = payment.groupby('provider_id')['customer_id'].nunique()

multiple_customers_providers = provider_counts[provider_counts > 1]

if len(multiple_customers_providers) > 0:
    print("There are provider_ids with multiple customers")
    print(multiple_customers_providers)
else:
    print("There are no provider_id with multiple customers")  
'''

'\nprovider_counts = payment.groupby(\'provider_id\')[\'customer_id\'].nunique()\n\nmultiple_customers_providers = provider_counts[provider_counts > 1]\n\nif len(multiple_customers_providers) > 0:\n    print("There are provider_ids with multiple customers")\n    print(multiple_customers_providers)\nelse:\n    print("There are no provider_id with multiple customers")  \n'

In [109]:
# Group the payment dataframe by customer_id and get the latest payment_status_id for each customer
latest_payment_status = payment.groupby('customer_id')['payment_status_id'].last()

# Create a new column 'payment_status_id' in rfm_dataset and fill it with the latest payment_status_ids
rfm_data['payment_status_id'] = rfm_data['customer_id'].map(latest_payment_status)


###  Step 1 - > Recency

In [111]:
#we will count information until today
reference_date = datetime.datetime.today().date()

In [112]:
# creating extra columns
rfm_data['days_since_last_call'] = (pd.to_datetime(reference_date) - pd.to_datetime(rfm_data['last_call'])).astype('timedelta64[D]')
rfm_data['days_since_last_touch'] = (pd.to_datetime(reference_date) - pd.to_datetime(rfm_data['last_touch'])).astype('timedelta64[D]')
rfm_data['days_since_last_seen'] = (pd.to_datetime(reference_date) - pd.to_datetime(rfm_data['last_seen_at'])).astype('timedelta64[D]')

In [113]:
# Fill NaN values in 'days_since_last_call' with the maximum value from the column
max_last_call = rfm_data['days_since_last_call'].max()
rfm_data['days_since_last_call'].fillna(max_last_call, inplace=True)

# Fill NaN values in 'days_since_last_touch' with the maximum value from the column
max_last_touch = rfm_data['days_since_last_touch'].max()
rfm_data['days_since_last_touch'].fillna(max_last_touch, inplace=True)

In [114]:
columns_to_normalize = ['budget_value', 'lead_read_gap_min', 'lead_count','view_count','image_count','video_count','discount_count','review_count','touch_count','call_count']  # List of columns to normalize

for column in columns_to_normalize:
    min_val = rfm_data[column].min()
    max_val = rfm_data[column].max()
    rfm_data[column] = (rfm_data[column] - min_val) / (max_val - min_val)

### Step 2 - > Frequency Part-Monetary Part
##### For **frequency**, we can use the following  : *lead_count*,*view_count*, *call_count* and *touch_count* and for **Monetary** our necessary columns will be : *budget_value*,*current_product_price*

In [115]:
rfm_data = rfm_data[['provider_id', 'customer_id','lead_count','view_count','image_count','video_count','discount_count','review_count','touch_count','call_count',
                    'membership_length','budget_value','current_products_price','lead_read_gap_min',
                    'days_since_last_call', 'days_since_last_touch','days_since_last_seen','satisfaction_status','payment_status_id']]

In [116]:
#our finalized dataframe to work on can be seen here: 
rfm_data.head(2)

,provider_id,customer_id,lead_count,view_count,image_count,video_count,discount_count,review_count,touch_count,call_count,membership_length,budget_value,current_products_price,lead_read_gap_min,days_since_last_call,days_since_last_touch,days_since_last_seen,satisfaction_status,payment_status_id
2,117,2253,0.324134,0.444696,0.080592,0.059140,0.000000,0.147297,0.108333,0.147059,96,0.023067,51288.0,0.031348,140.0,82.0,63.0,Memnun,future_payment
6,143,2254,0.112965,0.085073,0.073191,0.005376,0.222222,0.050000,0.050000,0.088235,108,0.023067,110664.0,0.005032,75.0,75.0,63.0,Memnun,future_payment


In [117]:
# Function to calculate Monetary column
def calculate_monetary(df):
    """
    this function will be used to create a monetary score and assign it to a seperate column created and named as Monetary
    """
    df['Monetary'] = df['current_products_price'] * df['budget_value']

# Function to calculate Frequency column
def calculate_frequency(df):
    """
    this function will be used to create a frequency score and assign it to a seperate column created and named as Frequency
    """
    df['Frequency'] = (df['image_count'] + df['video_count'] + df['discount_count'] + df['review_count'] + df['lead_count'] + df['view_count'] +
                      (1.5 * (df['touch_count'] + df['call_count']))) / df['membership_length']

# Function to calculate Recency column
def calculate_recency(df):
    """
    this function will be used to create a recency score and assign it to a seperate column created and named as Recency
    """
    min_last_touch = df['days_since_last_touch'].min()
    min_last_seen = df['days_since_last_seen'].min()
    min_last_call = df['days_since_last_call'].min()

    df['Recency'] = np.minimum.reduce([min_last_touch, min_last_seen, min_last_call]) * df['lead_read_gap_min']


In [118]:
# Calculate the columns using the defined functions
calculate_monetary(rfm_data)
calculate_frequency(rfm_data)
calculate_recency(rfm_data)

In [119]:
#now let's create our final RFM dataset to evaluate :
rfm_providers=rfm_data[["provider_id"]]
rfm_customers= rfm_data[["customer_id"]]
rfm_satisfaction=rfm_data[["satisfaction_status"]]
rfm_payment=rfm_data[["payment_status_id"]]
rfm_data = rfm_data[["Recency","Monetary","Frequency"]]

In [120]:
quantiles = rfm_data.quantile(q=[0.25,0.5,0.75])
quantiles

,Recency,Monetary,Frequency
0.25,0.214508,94.391860,0.008535
0.50,1.729155,322.475307,0.016372
0.75,8.648684,1392.868686,0.032273


In [121]:
quantiles.to_dict()

{'Recency': {0.25: 0.21450831024930747,
  0.5: 1.7291551246537396,
  0.75: 8.648684210526316},
 'Monetary': {0.25: 94.39186042933774,
  0.5: 322.47530743905065,
  0.75: 1392.8686858326753},
 'Frequency': {0.25: 0.008535109955235395,
  0.5: 0.016371864622555635,
  0.75: 0.03227345905319448}}

In [122]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [123]:
rfm_segmentation = rfm_data
rfm_segmentation['R_Quartile'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
rfm_segmentation['M_Quartile'] = rfm_segmentation['Monetary'].apply(FMScore, args=('Monetary',quantiles,))

In [124]:
rfm_segmentation.head()

,Recency,Monetary,Frequency,R_Quartile,F_Quartile,M_Quartile
2,1.974931,1183.055334,0.014989,2,2,3
6,0.317036,2552.675781,0.007002,3,1,4
7,0.069806,168.884932,0.013349,4,2,2
9,0.301766,677.585534,0.008735,3,2,3
10,1.651350,5060.786306,0.008125,3,1,4


In [125]:
rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \
                            + rfm_segmentation.F_Quartile.map(str) \
                            + rfm_segmentation.M_Quartile.map(str)
rfm_segmentation.head()

,Recency,Monetary,Frequency,R_Quartile,F_Quartile,M_Quartile,RFMScore
2,1.974931,1183.055334,0.014989,2,2,3,223
6,0.317036,2552.675781,0.007002,3,1,4,314
7,0.069806,168.884932,0.013349,4,2,2,422
9,0.301766,677.585534,0.008735,3,2,3,323
10,1.651350,5060.786306,0.008125,3,1,4,314


In [126]:
rfm_segmentation['RFMScore_num'] = rfm_segmentation.R_Quartile \
                            + rfm_segmentation.F_Quartile \
                            + rfm_segmentation.M_Quartile
rfm_segmentation.head()

,Recency,Monetary,Frequency,R_Quartile,F_Quartile,M_Quartile,RFMScore,RFMScore_num
2,1.974931,1183.055334,0.014989,2,2,3,223,7
6,0.317036,2552.675781,0.007002,3,1,4,314,8
7,0.069806,168.884932,0.013349,4,2,2,422,8
9,0.301766,677.585534,0.008735,3,2,3,323,8
10,1.651350,5060.786306,0.008125,3,1,4,314,8


In [127]:
dfs=[rfm_providers,rfm_customers,rfm_satisfaction,rfm_payment,rfm_segmentation]
for df in dfs:
    df.reset_index(drop=True, inplace=True)

In [129]:
merged_df = dfs[0]

# Merge the remaining dataframes on index one by one
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df, left_index=True, right_index=True)

In [131]:
#Saving the feature engineering results as CSV file
merged_df.to_csv(Path("..","data","rfm_segmentation.csv"),index=False)